In [1]:
import Pkg; Pkg.activate(joinpath(@__DIR__, "..")); Pkg.instantiate();
using Altro
using TrajectoryOptimization
using RobotDynamics
using StaticArrays
using LinearAlgebra
using Rotations
const TO = TrajectoryOptimization

TrajectoryOptimization

# Rocket MPC
In this problem we'll walk through how to set up a simple rocket model, including second-order cone constraints, generate a reference trajectory, and then track that reference trajectory with Conic MPC to solve the soft-landing problem. We will also cover how to define your own constraints.


## 1. Defining the Model
For this problem we'll use a simple linear model of a rocket flying vertically. This simplified model is a decent approximation when the roll angle is small.
This exposes part of the API for linear models defined by RobotDynamics.jl. The call to `LinearizedModel` can be used with any model, including nonlinear ones, to generate a discrete linear or linear-affine model.

In [31]:
function RocketModel(mass, grav, dt, ωPlanet = [0.0; 0.0; 0.0];
        integration=RobotDynamics.Exponential)
    
    # Define the dynamics matrices (including the affine term)
    A = [
        zeros(3,3)      I(3);
        -Rotations.skew(ωPlanet)^2   -2*Rotations.skew(ωPlanet);
    ]
    B = [
        zeros(3,3);
        1/mass * I(3)
    ]
    d = [
        zeros(3);
        grav
    ]

    # Continuous Linear-Affine Model
    cmodel =  LinearModel(A,B,d)

    # Discrete Model
    model = LinearizedModel(cmodel, dt=dt, is_affine=true, integration=integration)
end

# Generate the model
mass = 10.0             # kg
ωPlanet = [0, 0, 0]     # rad/s
gravity = [0, 0, -9.81]  # m/s/s
N = 301                 # number of knot points
dt = 0.05               # sec
tf = (N-1)*dt           # sec
model = RocketModel(mass, gravity, dt, ωPlanet);
n,m = size(model)

# TIP: Get the A,B matrices for the discrete model
RobotDynamics.get_A(model)
RobotDynamics.get_B(model);

## 2. Define the Objective

In [32]:
Q = Diagonal(@SVector fill(1e-2, n))
R = Diagonal(@SVector fill(1e-0, m))
Qf = Diagonal(@SVector fill(1e4, n))
xf = @SVector zeros(n)
x0 = SA_F64[4.0, 2.0, 20.0, -3.0, 2.0, -5.0]  # km
obj = LQRObjective(Q, R, Qf, xf, N);

## 3. Define the Constraints
For the constraints, we'll start off with the ones that we can define using the constraints [already defined in TrajectoryOptimization.jl](http://roboticexplorationlab.org/TrajectoryOptimization.jl/dev/constraint_api.html#Implemented-Constraints).

In [33]:
cons = ConstraintList(n, m, N)

# Goal Constraint
TO.add_constraint!(cons, GoalConstraint(xf), N)

# Maximum Thrust
# ||u||₂ <= u_bnd
perWeightMax = 2.0  
u_bnd = mass * abs(gravity[3]) * perWeightMax
maxThrust = NormConstraint(n, m, u_bnd, TO.SecondOrderCone(), :control)
TO.add_constraint!(cons, maxThrust, 1:N-1)

### Defining a Custom Constraint
For the next constraints, we need a custom constraint that can handle a more general SOCP constraint. We'll define a constraint of the form:
$$ || A z ||_2 \leq c^T z $$
where $z$ is the concatenated vector of the states and controls.

For a second-order cone constaint of the form $||v||_2 \leq c$, we define the output of the constraint to be $\begin{bmatrix} v^T & c \end{bmatrix}^T$. The Jacobian of the constraint is then just the Jacobian of this vector with respect to the inputs.

When defining our constraint below, we specify that it is a subclass of `StageConstraint`, meaning it is a function of the states and controls of a single knot point. Most other constraints will either be `StateConstraint` or `ControlConstraint`. This just tells the solver how wide the non-zero block in the constraint Jacobian is. Additionally, we need to provide methods for querying the state, control, and constraint output dimensions. Then, obviously, we need to define the constraint function via the `evaluate` method, which can be of any of the following forms:
* `evaluate(con, z::KnotPoint)` - Stage, state, and control constraints
* `evaluate(con, x::StaticVector, u::StaticVector)` - Stage constraints
* `evaluate(con, x::StaticVector)` - State constraints
* `evaluate(con, u::ControlVector)` - Control constraints

Since it's convenient to index out of the concatentated state and control vector stored in the `KnotPoint` type, we use the first method. 

We also need to define the Jacobian via the `jacobian!` method. Jacobians are only automatically defined using [ForwardDiff.jl](https://github.com/JuliaDiff/ForwardDiff.jlhttps://github.com/JuliaDiff/ForwardDiff.jl) for state and control constraints.

In [34]:
# Is a sub-class of StageConstraint since it is a function of both the states and controls
struct LinearNormConstraint{N,D,ND} <: TO.StageConstraint
    n::Int  # state dimension
    m::Int  # control dimension
    A::SMatrix{D, N, Float64, ND}
    c::SVector{N, Float64}
    inds::SVector{N, Int}  # extracts out a portion of the z vector
    function LinearNormConstraint{N,D}(n::Int, m::Int, A::AbstractMatrix, c::AbstractVector, inds=SVector{n+m}(1:n+m)) where {N,D}
        if inds == :state
            inds = SVector{n}(1:n)
        elseif inds == :control
            inds = SVector{m}(n .+ (1:m))
        end
        new{N,D,N*D}(n, m, SMatrix{D,N}(A), SVector{N}(c), inds)
    end
end
TO.state_dim(con::LinearNormConstraint) = con.n
TO.control_dim(con::LinearNormConstraint) = con.m
TO.sense(::LinearNormConstraint) = TO.SecondOrderCone()
TO.Base.length(::LinearNormConstraint{<:Any,D}) where D = D + 1

function TO.evaluate(con::LinearNormConstraint, z::KnotPoint)
    x = z.z[con.inds]
    v = con.A*x
    s = con.c'x
    return push(v, s)  # stack the vector and scalar parts
end

function TO.jacobian!(∇c, con::LinearNormConstraint{N,D}, z::KnotPoint) where {N,D}
    ∇c[1:D,con.inds] .= con.A
    ∇c[D+1,con.inds] .= con.c
end

In [35]:
# Max Angle angle
# || [u_x, u_y] || ≤ tan(θ) u_z
θ_thrust_max = 5.0 # deg
α_max = tand(θ_thrust_max)

AThrust = SA_F64[
    1 0 0;
    0 1 0;
    0 0 0
]
cThrust = SA_F64[0, 0, α_max]
maxAngle = LinearNormConstraint{3,3}(n, m, AThrust, cThrust, :control)
TO.add_constraint!(cons, maxAngle, 1:N-1)

# Glideslope Constraint
# Keeps the rocket within a conic airspace, with the tip at the landing site
# Keeps the rocket from flying low to the ground if it's not near the landing site
θ_glideslope = 45 # deg
α_glide = tand(θ_glideslope)
AGlide = SA_F64[
    1 0 0;
    0 1 0;
    0 0 0;
]
cGlide = SA_F64[0, 0, α_glide]
glideSlope = LinearNormConstraint{3,3}(n, m, AGlide, cGlide, SA[1,2,3])
TO.add_constraint!(cons, glideSlope, 1:N-1)

## 3. Define the problem
We can now define our problem. Since our dynamics are already discretized, we want to disable the automatic integration by setting the integration method to `RobotDynamics.PassThrough`.

In [45]:
prob = Problem(model, obj, xf, tf, x0=x0, constraints=cons, integration=RobotDynamics.PassThrough);
U0 = [-mass * gravity for k = 1:N-1]
initial_controls!(prob, U0)
rollout!(prob)

PassThrough

In [40]:
opts = SolverOptions(
    cost_tolerance_intermediate=1e-4,
    penalty_scaling=500.,
    penalty_initial=1e-2,
#     verbose = 1,
    projected_newton = false,
    constraint_tolerance = 1.0e-5,
    iterations = 5000,
    iterations_inner = 100,
    iterations_linesearch = 100,
    iterations_outer = 500,
)
solver = ALTROSolver(prob, opts)
# solve!(solver)

LoadError: AssertionError: Can't call continuous dynamics on a discrete LinearModel

In [43]:
F = zeros(n, n+m)
discrete_jacobian!(RobotDynamics.PassThrough, F, model, z)